## Post-process PROTEUS output with high resolution radiative transfer
Note that this is a Julia notebook, not a Python one.

In [1]:
# Activate environment
PROTEUS_DIR = abspath(joinpath(pwd(), "../"))
ROOT_DIR = abspath( PROTEUS_DIR , "AGNI/")
using Pkg
Pkg.activate(ROOT_DIR)

# Import system packages
using Printf
using Plots
using LaTeXStrings
using NCDatasets
using Glob

# Import AGNI
using AGNI
import AGNI.atmosphere as atmosphere
import AGNI.energy as energy
import AGNI.dump as dump
import AGNI.plotting as plotting
import AGNI.setpt as setpt


  Activating project at `/dataserver/users/formingworlds/nicholls/PROTEUS/AGNI`


In [2]:
# Set simulation output folder
output_dir = joinpath(PROTEUS_DIR, "output", "test")

"/dataserver/users/formingworlds/nicholls/PROTEUS/output/test"

In [8]:
# use same spectral file as simulation
spectral_file = joinpath(output_dir, "runtime.sf")
star_file = ""

# use high resolution file
spectral_file = joinpath(ENV["FWL_DATA"], "spectral_files/Honeyside/4096/Honeyside.sf")
star_file = joinpath(output_dir, "data", "0.sflux")

if !ispath(spectral_file)
    error("Cannot find spectral file $spectral_file")
end

# read model output
files = glob("*_atm.nc", joinpath(output_dir , "data"))

# drop files
files = files[1:50]

nfiles = length(files)
@printf("Found %d files in output folder", nfiles)

Found 50 files in output folder

In [9]:
# Setup initial atmos struct...
fpath = files[1]
@printf("Setup atmos from %s", fpath)

ds = Dataset(fpath,"r")

# Get all of the information that we need
nlev_c::Int = length(ds["p"][:])

#   temperature arrays
input_tmpl::Array{Float64,1} = ds["tmpl"][:]
input_tmp::Array{Float64,1}  = ds["tmp"][:]

#   pressure arrays
input_pl::Array{Float64,1} = ds["pl"][:]
input_p::Array{Float64,1}  = ds["p"][:]

#   gas names
raw_gases::Array{Char,2} = ds["gases"][:,:]
num_gas::Int = size(raw_gases)[2]
input_gases::Array{String,1} = []
for i in 1:num_gas
    push!(input_gases, strip(String(raw_gases[:,i])))
end

# gas VMRs
raw_vmrs::Array{Float64, 2} = ds["x_gas"][:,:]
input_vmrs::Dict{String, Array{Float64,1}} = Dict()      # dict of Arrays
input_vmrs_scalar::Dict{String, Float64} = Dict()        # dict of Floats (surface values)
for i in 1:num_gas
    g = input_gases[i]
    input_vmrs[g]     = zeros(Float64, nlev_c)
    input_vmrs[g][:] .= raw_vmrs[i, :]

    input_vmrs_scalar[g] = input_vmrs[g][end]
end

# surface
input_tsurf::Float64   = ds["tmp_surf"][1]
input_radius::Float64  = ds["planet_radius"][1]
input_gravity::Float64 = ds["surf_gravity"][1]

# stellar properties
input_inst::Float64   = ds["instellation"][1]
input_s0fact::Float64 = ds["inst_factor"][1]
input_albedo::Float64 = ds["bond_albedo"][1]
input_zenith::Float64 = ds["zenith_angle"][1]

# flags
input_flag_rayleigh::Bool  = Bool(ds["flag_rayleigh"][1] == 'y')
input_flag_thermo::Bool    = Bool(ds["thermo_funct"][1] == 'y')
input_flag_continuum::Bool = Bool(ds["flag_continuum"][1] == 'y')

# Close file
close(ds);

# Setup atmosphere
atmos = atmosphere.Atmos_t()
atmosphere.setup!(atmos, ROOT_DIR, output_dir,
                        spectral_file,
                        input_inst, input_s0fact, input_albedo, input_zenith,
                        input_tsurf,
                        input_gravity, input_radius,
                        nlev_c, input_pl[end], input_pl[1],
                        input_vmrs_scalar, "",
                        flag_gcontinuum=input_flag_continuum,
                        flag_rayleigh=input_flag_rayleigh,
                        thermo_functions=input_flag_thermo,
                        overlap_method=2
                        )
code = atmosphere.allocate!(atmos, star_file)
if !code
    error("Failed")
end

Setup atmos from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/0_atm.nc

┌ Info: Composition set by dict
└ @ AGNI.atmosphere /dataserver/users/formingworlds/nicholls/PROTEUS/AGNI/src/atmosphere.jl:460
┌ Info: Inserting stellar spectrum and Rayleigh coefficients
└ @ AGNI.spectrum /dataserver/users/formingworlds/nicholls/PROTEUS/AGNI/src/spectrum.jl:201
┌ Info: Allocated atmosphere with composition:
└ @ AGNI.atmosphere /dataserver/users/formingworlds/nicholls/PROTEUS/AGNI/src/atmosphere.jl:1197
┌ Info:       1 N2      3.05e-03 
└ @ AGNI.atmosphere /dataserver/users/formingworlds/nicholls/PROTEUS/AGNI/src/atmosphere.jl:1215
┌ Info:       2 CO      8.08e-01 
└ @ AGNI.atmosphere /dataserver/users/formingworlds/nicholls/PROTEUS/AGNI/src/atmosphere.jl:1215
┌ Info:       3 CH4     1.21e-06 
└ @ AGNI.atmosphere /dataserver/users/formingworlds/nicholls/PROTEUS/AGNI/src/atmosphere.jl:1215
┌ Info:       4 S2      2.19e-07 (NO_OPACITY)
└ @ AGNI.atmosphere /dataserver/users/formingworlds/nicholls/PROTEUS/AGNI/src/atmosphere.jl:1215
┌ Info:       5 SO2     9.16e-06 
└ @ A

In [10]:
function update_atmos_from_nc!(atmos, fpath)

    ds = Dataset(fpath,"r")
    @printf("    updating from %s \n", fpath)

    #   gas names
    raw_gases::Array{Char,2} = ds["gases"][:,:]
    num_gas::Int = size(raw_gases)[2]
    input_gases::Array{String,1} = []
    for i in 1:num_gas
        push!(input_gases, strip(String(raw_gases[:,i])))
    end

    # gas VMRs
    raw_vmrs::Array{Float64, 2} = ds["x_gas"][:,:]
    input_vmrs::Dict{String, Array{Float64,1}} = Dict()      # dict of Arrays
    input_vmrs_scalar::Dict{String, Float64} = Dict()        # dict of Floats (surface values)
    for i in 1:num_gas
        g = input_gases[i]
        input_vmrs[g]     = zeros(Float64, nlev_c)
        input_vmrs[g][:] .= raw_vmrs[i, :]

        input_vmrs_scalar[g] = input_vmrs[g][end]
    end


    # Update the struct with new values
    atmos.instellation =  ds["instellation"][1]

    atmos.tmp_surf = ds["tmp_surf"][1]
    atmos.tmp_magma = ds["tmagma"][1]
    atmos.tmpl[:] .= ds["tmpl"][:]
    atmos.tmp[:] .=  ds["tmp"][:]

    atmos.pl[:] .= ds["pl"][:]
    atmos.p[:] .=  ds["p"][:]
    atmos.p_boa = atmos.pl[end]

    for g in input_gases
        atmos.gas_vmr[g][:] .= input_vmrs[g]
    end

    atmosphere.calc_layer_props!(atmos)

    # Close file
    close(ds);

end

update_atmos_from_nc! (generic function with 1 method)

In [11]:
atm_arr = atmosphere.Atmos_t[]

# Loop over netcdfs and post-process them
for (i,fpath) in enumerate(files)

    # progress
    @printf("%3d /%3d = %.1f%% \n", i, length(files), i*100.0/length(files))

    # set new composition and structure
    update_atmos_from_nc!(atmos, fpath)

    # set fluxes to zero
    energy.reset_fluxes!(atmos)

    # do radtrans with this composition
    energy.radtrans!(atmos, true)   # LW
    energy.radtrans!(atmos, false)  # SW

    # store result
    push!(atm_arr, deepcopy(atmos))
end

  1 / 50 = 2.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/0_atm.nc 
  2 / 50 = 4.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/101579_atm.nc 
  3 / 50 = 6.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/1016656_atm.nc 
  4 / 50 = 8.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/1032959_atm.nc 
  5 / 50 = 10.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/10390_atm.nc 
  6 / 50 = 12.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/1050893_atm.nc 
  7 / 50 = 14.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/106454_atm.nc 
  8 / 50 = 16.0% 
    updating from /dataserver/users/formingworlds/nicholls/PROTEUS/output/test/data/1070621_atm.nc 
  9 / 50 = 18.0% 
    updating from /dataserver/users/formingworlds/ni

In [ ]:
# write new output files
for (i,fpath) in enumerate(files)
    @printf("%3d /%3d = %.1f%% \n", i, length(files), i*100.0/length(files))

    fpath = replace(fpath, "_atm.nc" => "_ppr.nc")
    dump.write_ncdf(atm_arr[i], joinpath(output_dir, "data", fpath))
end